In [2]:
import pandas as pd
import numpy as np
import re
import time
import json
import bs4 as bs4
import tqdm #tqdm é usando para mostrar barra de carregamento
import glob

pd.set_option("max.columns", 131)

#https://strtime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(498, 16)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [5]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

# 1. Limpeza da data

In [6]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)") #criando df com as 3 colunas
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x) #Adicionando 0 a frente dos numeros < 10

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1) #Aplicando a junção das colunas a cada linha

df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y") #Transforma a string como objeto data

# 2. Limpeza do Views

In [7]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

#Extract=False retorna uma série e não um df como padrão.

# 3. Features

In [8]:
features = pd.DataFrame(index=df_limpo.index) #mesmo indice do df_limpo
y = df['y'].copy()

In [9]:
features['tempo_desde_pub'] = (pd.to_datetime("2020-04-19") - df_limpo['date']) / np.timedelta64(1, 'D') #diferença de um dia
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [10]:
features.head()

,views,views_por_dia
0,28028,47.185185
1,1131,2.175000
2,1816,5.144476
3,1171,4.684000
4,1228,2.426877


In [11]:
#dividindo os dados de treino e validação

mask_train = df_limpo['date'] < "2019-04-01"
mask_val = df_limpo['date'] >= "2019-04-01"

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (270, 2), (228,), (270,))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2) #numero mínimo que uma palavra precisa aparecer nos dados para que se torne uma coluna 
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val) #Transforma usando apenas as palavras aprendidas no df de treino

In [13]:
title_bow_train.shape

(228, 193)

In [14]:
1-1277/(228*193)
# porcentagem dos elementos = 0

0.9709799109171894

In [15]:
title_bow_train
#Matriz sparse só armazena (memória) os elementos diferente de 0

<228x193 sparse matrix of type '<class 'numpy.float64'>'
	with 1277 stored elements in Compressed Sparse Row format>

In [16]:
#hstack - [1 2]  [3 4] -> [1 2 3 4]
#vstack - [1 2]  [3 4] -> [1 2]
#                         [3 4]

In [17]:
Xtrain.head()

,views,views_por_dia
0,28028,47.185185
1,1131,2.175000
4,1228,2.426877
6,430097,700.483713
7,88592,208.451765


In [18]:
from scipy.sparse import hstack, vstack

#Junto os dados com a matriz sparce
Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [19]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((228, 195), (270, 195))

In [20]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [21]:
p = mdl.predict_proba(Xval_wtitle)[:,1]

In [22]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [23]:
average_precision_score(yval, p)

0.18851986577534413

In [24]:
roc_auc_score(yval, p)

0.5814589665653496

# 5. Active learning

70 exemplos que o modelo tenha difilculdade
30 exemplos eleatoriamente

In [3]:
df_unlabeled = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all') # Remove linha onde todos os campos estão null 
df_unlabeled.shape

(674, 16)

In [6]:
df_unlabeled = df_unlabeled[df_unlabeled['y'].isnull()].dropna(how='all') # Remove linha onde todos os campos estão null 

In [7]:
df_unlabeled

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w
502,OpenAI Gym and Python for Q-learning - Reinfor...,NaN,20.378 visualizações,Publicado em 14 de out. de 2018,Educação,OpenAI Gym and Python for Q-learning - Reinfor...,deeplizard\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,20.378 visualizações\n\n\n\n\n\n\n\n397\n\nGos...,https://i.ytimg.com/vi/QK_PP_2KgGE/maxresdefau...,1280.0,720.0,Welcome back to this series on reinforcement l...,1280.0,720.0,experience replay,/channel/UC4UJ26WkceqONNF5S26OiVw
503,"Dashboarding with Notebooks, Day 1: What infor...",NaN,10.435 visualizações,Transmitido ao vivo em 17 de dez. de 2018,Ciência e tecnologia,"Dashboarding with Notebooks, Day 1: What infor...",Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,10.435 visualizações\n\n\n\n\n\n\n\n261\n\nGos...,https://i.ytimg.com/vi/QO2ihJS2QLM/maxresdefau...,1280.0,720.0,This livestream is for the first day of the Ka...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA
504,How To Get US- American Company H1 Visa To Get...,NaN,7 visualizações,Publicado em 23 de nov. de 2019,Pessoas e blogs,How To Get US- American Company H1 Visa To Get...,Rakesh Yadav Ki Baatain\n\n\n\n\n\n\n\n\n\n\n\...,7 visualizações\n\n\n\n\n\n\n\n0\n\nGostou des...,https://i.ytimg.com/vi/QOLa65_tjn8/maxresdefau...,1280.0,720.0,,960.0,720.0,NaN,/channel/UCcLYXkT37EAs0Ocn1WZ2OxQ
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,Kaggle Live Coding : Visualizing Clusters | Ka...,NaN,2.200 visualizações,Transmitido ao vivo em 16 de ago. de 2019,Ciência e tecnologia,Kaggle Live Coding : Visualizing Clusters | Ka...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,2.200 visualizações\n\n\n\n\n\n\n\n83\n\nGosto...,https://i.ytimg.com/vi/zTwL-oF9Kcc/hqdefault.jpg,480.0,360.0,This week we'll be figuring out how to efficie...,1280.0,720.0,reading group,/channel/UCSNeZleDn9c74yQc-EKnVTA
1180,"""The Power and Limits of Deep Learning"" with Y...",NaN,5.055 visualizações,Publicado em 11 de set. de 2019,Ciência e tecnologia,"""The Power and Limits of Deep Learning"" with Y...",Association for Computing Machinery (ACM)\n\n\...,5.055 visualizações\n\n\n\n\n\n\n\n133\n\nGost...,https://i.ytimg.com/vi/zikdDOzOpxY/hqdefault.jpg,480.0,360.0,"Title: The Power and Limits of Deep Learning"" ...",1280.0,720.0,Facebook,/channel/UCPyA0XmU6aS4JCwVoIBTmIQ
1181,Основы Python для Data Science,NaN,8.967 visualizações,Transmitido ao vivo em 1 de abr. de 2019,Entretenimento,#основы_Python_для_Data_Science #что_такое_Dat...,Skillbox - Программирование\n\n\n\n\n\n\n\n\n\...,8.967 visualizações\n\n\n\n\n\n\n\n241\n\nGost...,https://i.ytimg.com/vi/zqcIjDLxZIM/maxresdefau...,1280.0,720.0,Положи начало профессии мечты с курсом ⇒ http:...,1280.0,720.0,язык python с нуля,/channel/UCHJZFCpwlXV7Sie1dV6pQLw
1182,GOTO 2019 • Composing Bach Chorales Using Deep...,NaN,938 visualizações,Transmitido ao vivo em 18 de nov. de 2019,Ciência e tec

In [27]:
df_unlabeled.head(2)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w
502,OpenAI Gym and Python for Q-learning - Reinfor...,NaN,20.378 visualizações,Publicado em 14 de out. de 2018,Educação,OpenAI Gym and Python for Q-learning - Reinfor...,deeplizard\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,20.378 visualizações\n\n\n\n\n\n\n\n397\n\nGos...,https://i.ytimg.com/vi/QK_PP_2KgGE/maxresdefau...,1280.0,720.0,Welcome back to this series on reinforcement l...,1280.0,720.0,experience replay,/channel/UC4UJ26WkceqONNF5S26OiVw


In [28]:
df_limpo_u = pd.DataFrame(index=df_unlabeled.index)
df_limpo_u['title'] = df_unlabeled['watch-title']

In [29]:
clean_date = df_unlabeled['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)") #criando df com as 3 colunas
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x) #Adicionando 0 a frente dos numeros < 10

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1) #Aplicando a junção das colunas a cada linha

df_limpo_u['date'] = pd.to_datetime(clean_date, format="%d %b %Y") #Transforma a string como objeto data

In [30]:
df_limpo_u.head()

,title,date
501,Kaggle Mercari Price Suggestion Challenge (1 p...,2018-11-02
502,OpenAI Gym and Python for Q-learning - Reinfor...,2018-10-14
503,"Dashboarding with Notebooks, Day 1: What infor...",2018-12-17
504,How To Get US- American Company H1 Visa To Get...,2019-11-23
505,Platform Overview - Machine Learning,2019-05-21


In [31]:
views = df_unlabeled['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo_u['views'] = views

#Extract=False retorna uma série e não um df como padrão.

In [32]:
features_u = pd.DataFrame(index=df_limpo_u.index) #mesmo indice do df_limpo_u

In [35]:
features_u['tempo_desde_pub'] = (pd.to_datetime("2020-04-19") - df_limpo_u['date']) / np.timedelta64(1, 'D') #diferença de um dia
features_u['views'] = df_limpo_u['views']
features_u['views_por_dia'] = features_u['views'] / features_u['tempo_desde_pub']
features_u = features_u.drop(['tempo_desde_pub'], axis=1)

In [36]:
features_u.head()

,views,views_por_dia
501,2167,4.058052
502,20378,36.849910
503,10435,21.339468
504,7,0.047297
505,4298,12.868263


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_u = df_limpo_u['title']
title_bow_u = title_vec.transform(title_u)

In [38]:
title_bow_u

<674x193 sparse matrix of type '<class 'numpy.float64'>'
	with 3079 stored elements in Compressed Sparse Row format>

In [40]:
Xu_wtitle = hstack([features_u, title_bow_u])
Xu_wtitle

<674x195 sparse matrix of type '<class 'numpy.float64'>'
	with 4409 stored elements in COOrdinate format>

In [41]:
pu = mdl.predict_proba(Xu_wtitle)[:,1]

In [42]:
df_unlabeled['p'] = pu

In [43]:
df_unlabeled.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
501,Kaggle Mercari Price Suggestion Challenge (1 p...,NaN,2.167 visualizações,Publicado em 2 de nov. de 2018,Educação,Kaggle Mercari Price Suggestion Challenge (1 p...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,2.167 visualizações\n\n\n\n\n\n\n\n61\n\nGosto...,https://i.ytimg.com/vi/QFR0IHbzA30/maxresdefau...,1280.0,720.0,Pawel Jankiewicz and Konstantin Lopuhin share ...,1280.0,720.0,price suggestion,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w,0.087


In [54]:
#Selecionando 70 exemplos confusos para o modelo
mask_u = (df_unlabeled['p'] >= 0.26) & (df_unlabeled['p'] <= 1.)
mask_u.sum()

70

In [53]:
df_unlabeled[mask_u].sort_values("p")

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
582,Kaggle Reading Group: XLNet | Kaggle,NaN,3.127 visualizações,Transmitido ao vivo em 3 de jul. de 2019,Ciência e tecnologia,Kaggle Reading Group: XLNet | Kaggle,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,3.127 visualizações\n\n\n\n\n\n\n\n89\n\nGosto...,https://i.ytimg.com/vi/Ubh03Sh45b0/hqdefault.jpg,480.0,360.0,This week we're starting a new paper in the Ka...,1280.0,720.0,reading group,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.261
978,Machine Learning with python Course - Lecture ...,NaN,2.107 visualizações,Publicado em 30 de mai. de 2019,Pessoas e blogs,#supervised #unsupervised #learning\n\n\n\n ...,Machine Learning With python\n\n\n\n\n\n\n\n\n...,2.107 visualizações\n\n\n\n\n\n\n\n52\n\nGosto...,https://i.ytimg.com/vi/otXTykwJsd4/maxresdefau...,1280.0,720.0,- You will Learn Machine Learning topics such ...,1280.0,720.0,#Algorithms,/channel/UCu1apYC0izYTG3X_QH_O4hw,0.262
519,flowlightさん『TalkingData AdTracking Fraud Detec...,NaN,1.707 visualizações,Publicado em 12 de mai. de 2018,Pessoas e blogs,flowlightさん『TalkingData AdTracking Fraud Detec...,Takami Sato\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,1.707 visualizações\n\n\n\n\n\n\n\n17\n\nGosto...,https://i.ytimg.com/vi/RKXUEJVJJ-o/maxresdefau...,1280.0,720.0,資料 https://www.slideshare.net/TakanoriHayashi3...,1280.0,720.0,kaggle,/channel/UCiECS_auJLNpFsvjTi1WuxQ,0.266
1007,Kaggle Reading Group: Deep Learning for Symbol...,NaN,1.532 visualizações,Transmitido ao vivo em 2 de out. de 2019,Ciência e tecnologia,Kaggle Reading Group: Deep Learning for Symbol...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.532 visualizações\n\n\n\n\n\n\n\n55\n\nGosto...,https://i.ytimg.com/vi/q_cMExRsJl8/hqdefault.jpg,480.0,360.0,"This week we start in on ""Deep Learning for Sy...",1280.0,720.0,reading group,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.266
966,"Si quiero dedicarme al Machine Learning, ¿qué ...",NaN,2.976 visualizações,Publicado em 5 de out. de 2019,Ciência e tecnologia,"Si quiero dedicarme al Machine Learning, ¿qué ...",Juan Gabriel Gomila Salas\n\n\n\n\n\n\n\n\n\n\...,2.976 visualizações\n\n\n\n\n\n\n\n168\n\nGost...,https://i.ytimg.com/vi/oKHOSqQw9ro/maxresdefau...,1280.0,720.0,Tienes todos los cursos citados en el video co...,960.0,720.0,estadistica,/channel/UCMUxXNYrVCv6-bQakhomvBg,0.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Kaggle Days Meetup Milan #2 - Luca Massaron,NaN,55 visualizações,Publicado em 25 de out. de 2019,Pessoas e blogs,Kaggle Days Meetup Milan #2 - Luca Massaron,LogicAI\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando....,55 visualizações\n\n\n\n\n\n\n\n3\n\nGostou de...,https://i.ytimg.com/vi/WlBRu6BE0AQ/maxresdefau...,1280.0,720.0,The second edition of Kaggle Days Meetup. We h...,1280.0,720.0,kaggle,/channel/UCZ_qlZbg9EzwRnLq_hFQumQ,0.593
938,Kaggle Meetup: Skin Cancer Diagnosis,NaN,663 visualizações,Publicado em 5 de out. de 2018,Ciência e tecnologia,Kaggle Meetup: Skin Cancer Diagnosis,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,663 visualizações\n\n\n\n\n\n\n\n22\n\nGostou ...,https://i.ytimg.com/vi/meg8I1GdtUg/maxresdefau...,1280.0,720.0,Video from this meetup: https://www.meetup.com...,1280.0,720.0,machine learning,/channel/UCJhW_16uxALr0X4olEW2p5A,0.613
908,Kaggle Meetup: Quora Insincere Questions Class...,NaN,399 visualizações,Publicado em 8 de mar. de 2019,Ciência e tecnologia,Kaggle Meetup: Quora Insincere Questions Class...,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,399 visualizações\n\n\n\n\n\n\n\n7\n\nGostou d...,https://i.ytimg.com/vi/l7ZmviwSnXw/maxresdefau...,1280.0,720.0,Kaggle page: https://www.kaggle.com/c/quora-in...,1280.0,720.0,artificial intelligence,/channel/UCJhW_16uxALr0X4olEW2p5A,0.655
919,Kaggle Days Meetup Milan #1,NaN,243 visualizações,Publicado

In [55]:
dificeis = df_unlabeled[mask_u]

In [58]:
aleatorios = df_unlabeled[~mask_u].sample(30) #Amostra de 30 exemplos que não são os elementos de "mask_u"

In [59]:
pd.concat([dificeis, aleatorios]).to_csv("active_label1.csv")

In [60]:
dificeis.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
505,Platform Overview - Machine Learning,NaN,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.520
507,Kaggle Meetup: Ship Detection Challenge,NaN,504 visualizações,Publicado em 30 de nov. de 2018,Ciência e tecnologia,Kaggle Meetup: Ship Detection Challenge,Learn Data Science\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,504 visualizações\n\n\n\n\n\n\n\n9\n\nGostou d...,https://i.ytimg.com/vi/QXEy4rdLsDw/maxresdefau...,1280.0,720.0,Video from the 2018-11-29 meetup. Kaggle page:...,1280.0,720.0,learn data science,/channel/UCJhW_16uxALr0X4olEW2p5A,0.454
519,flowlightさん『TalkingData AdTracking Fraud Detec...,NaN,1.707 visualizações,Publicado em 12 de mai. de 2018,Pessoas e blogs,flowlightさん『TalkingData AdTracking Fraud Detec...,Takami Sato\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarrega...,1.707 visualizações\n\n\n\n\n\n\n\n17\n\nGosto...,https://i.ytimg.com/vi/RKXUEJVJJ-o/maxresdefau...,1280.0,720.0,資料 https://www.slideshare.net/TakanoriHayashi3...,1280.0,720.0,kaggle,/channel/UCiECS_auJLNpFsvjTi1WuxQ,0.266
521,Kaggle iMaterialist (Fashion) 2019 at FGVC6 — ...,NaN,526 visualizações,Publicado em 18 de set. de 2019,Educação,Kaggle iMaterialist (Fashion) 2019 at FGVC6 — ...,ML Trainings\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarreg...,526 visualizações\n\n\n\n\n\n\n\n18\n\nGostou ...,https://i.ytimg.com/vi/RUfmEj1MC3k/maxresdefau...,1280.0,720.0,Илья Денисов рассказывает про опыт участия в с...,1280.0,720.0,kaggle,/channel/UCeq6ZIlvC9SVsfhfKnSvM9w,0.273
527,Reinforcement Learning with TensorFlow and Uni...,NaN,545 visualizações,Publicado em 22 de nov. de 2019,Ciência e tecnologia,Reinforcement Learning with TensorFlow and Uni...,Google Developers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...,545 visualizações\n\n\n\n\n\n\n\n13\n\nGostou ...,https://i.ytimg.com/vi/S-MbpQiwfls/maxresdefau...,1280.0,720.0,"Dan Goncharov, Head of 42 Robotics GDG Fremont...",1280.0,720.0,Dan Goncharov,/channel/UC_x5XG1OV2P6uZZ5FSM9Ttw,0.297


In [61]:
aleatorios.head()

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p
516,Kaggle Reading Group: XLNet (Part 4) | Kaggle,NaN,1.198 visualizações,Transmitido ao vivo em 31 de jul. de 2019,Ciência e tecnologia,Kaggle Reading Group: XLNet (Part 4) | Kaggle,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,1.198 visualizações\n\n\n\n\n\n\n\n47\n\nGosto...,https://i.ytimg.com/vi/RE4fzA5RKJM/hqdefault.jpg,480.0,360.0,This week we're continuing with XLNet: General...,1280.0,720.0,reading group,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.145
964,Big Data y Data Science,NaN,Nenhuma visualização,Estreou em 22 de nov. de 2019,Pessoas e blogs,Big Data y Data Science,Franco Castagnola\n\n\n\n\n\n\n\n\n\n\n\n\n\nC...,Nenhuma visualização\n\n\n\n\n\n\n\n0\n\nGosto...,https://i.ytimg.com/vi/oD6tBBdr9rc/maxresdefau...,1280.0,720.0,Video 4 academy.easywere.cl,1280.0,720.0,NaN,/channel/UCZrBfngWy_IkrmWTrN9ftXw,0.121
1034,Tricks to Reach Kaggle Leaderboard | by Daniji...,NaN,564 visualizações,Publicado em 20 de mai. de 2019,Ciência e tecnologia,Tricks to Reach Kaggle Leaderboard | by Daniji...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,564 visualizações\n\n\n\n\n\n\n\n15\n\nGostou ...,https://i.ytimg.com/vi/s3YkZPNKLYg/maxresdefau...,1280.0,720.0,"""Feature Engineering Tricks to Reach Kaggle Le...",1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.164
545,Intelligence Artificielle VS Machine Learning ...,NaN,6.803 visualizações,Publicado em 10 de out. de 2018,Ciência e tecnologia,Intelligence Artificielle VS Machine Learning ...,Computer Scientists\n\n\n\n\n\n\n\n\n\n\n\n\n\...,6.803 visualizações\n\n\n\n\n\n\n\n180\n\nGost...,https://i.ytimg.com/vi/SfOoRsUj9kQ/maxresdefau...,1280.0,720.0,Aujourd'hui on parles des différences qui rési...,1280.0,720.0,Computer Scientists,/channel/UChXy935RrhKDL3EtFcITM5w,0.246
1083,Kaggle Live-Coding: Deep Learning with GPUs (p...,NaN,2.576 visualizações,Publicado em 18 de jun. de 2018,Ciência e tecnologia,Kaggle Live-Coding: Deep Learning with GPUs (p...,Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregando.....,2.576 visualizações\n\n\n\n\n\n\n\n58\n\nGosto...,https://i.ytimg.com/vi/u9pIhOay8Fw/maxresdefau...,1280.0,720.0,Join Kaggle Data Scientist Rachael as she star...,1280.0,720.0,CS,/channel/UCSNeZleDn9c74yQc-EKnVTA,0.122
